📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/pytorch-workshop](https://github.com/mr-pylin/pytorch-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [VGGNet](#toc2_)    
  - [Custom VGGNet](#toc2_1_)    
    - [VGG11](#toc2_1_1_)    
      - [Initialize the Model](#toc2_1_1_1_)    
      - [Model Summary](#toc2_1_1_2_)    
    - [VGG13](#toc2_1_2_)    
      - [Initialize the Model](#toc2_1_2_1_)    
      - [Model Summary](#toc2_1_2_2_)    
    - [VGG16](#toc2_1_3_)    
      - [Initialize the Model](#toc2_1_3_1_)    
      - [Model Summary](#toc2_1_3_2_)    
    - [VGG19](#toc2_1_4_)    
      - [Initialize the Model](#toc2_1_4_1_)    
      - [Model Summary](#toc2_1_4_2_)    
  - [PyTorch VGGNet](#toc2_2_)    
    - [VGG11](#toc2_2_1_)    
      - [Initialize the Model](#toc2_2_1_1_)    
      - [Model Summary](#toc2_2_1_2_)    
    - [VGG13](#toc2_2_2_)    
      - [Initialize the Model](#toc2_2_2_1_)    
      - [Model Summary](#toc2_2_2_2_)    
    - [VGG16](#toc2_2_3_)    
      - [Initialize the Model](#toc2_2_3_1_)    
      - [Model Summary](#toc2_2_3_2_)    
    - [VGG19](#toc2_2_4_)    
      - [Initialize the Model](#toc2_2_4_1_)    
      - [Model Summary](#toc2_2_4_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

- torchvision models:
  - class
    - brings in the model class directly
    - Allows more control and customization since you are dealing directly with the class. You can override methods, customize initialization, etc.
  - function
    - This import brings in a function that returns an instance of the model
    - Easier and quicker to use, especially for standard models
- [docs.pytorch.org/vision/stable/models.html](https://docs.pytorch.org/vision/stable/models.html)


In [ ]:
import torch
from torch import nn
from torchinfo import summary
from torchvision.models import VGG, vgg11, vgg13, vgg16, vgg19

# <a id='toc2_'></a>[VGGNet](#toc0_)

- Developed in 2014 by [Karen Simonyan](https://dblp.uni-trier.de/search/author?author=Karen%20Simonyan) and [Andrew Zisserman](https://dblp.uni-trier.de/pid/z/AndrewZisserman.html?q=Andrew%20Zisserman) from the Visual Geometry Group ([VGG](https://www.robots.ox.ac.uk/~vgg/index.html)) at the University of [Oxford](https://www.ox.ac.uk/).
- It is based on the [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556) paper
- It was trained on the [ImageNet](https://www.image-net.org/) dataset (first resized to 256x256 then center cropped to 224x224) [[ImageNet viewer](https://navigu.net/#imagenet)]
- Known for its simple and uniform architecture, using small `3x3` convolutional filters consistently throughout the network
- It comes in several variants, primarily `VGG11`, `VGG13`, `VGG16` and `VGG19`, indicating the total number of layers
- The `runner-up` of the ImageNet Large Scale Visual Recognition Challenge ([ILSVRC](https://image-net.org/challenges/LSVRC/2014/)) in 2014

<figure style="text-align: center;">
  <img src="../../../assets/images/original/cnn/architectures/vgg16.svg" alt="vgg16-architecture.svg" style="width: 100%;">
  <figcaption>VGG16 Architecture</figcaption>
</figure>


## <a id='toc2_1_'></a>[Custom VGGNet](#toc0_)

- `Softmax` is missing due to internal implementation of `LogSoftmax` in the `CrossEntropyLoss` function.
- there is an extension of VGGNet which also contains `nn.BatchNorm2d` before `nn.ReLU` in the `feature_extractor` section.


In [ ]:
class CustomVGGNet(nn.Module):
    def __init__(self, feature_layers: list, num_classes: int = 1000):
        super().__init__()

        self.features = nn.Sequential(*self._make_layers(feature_layers))

        # 512x7x7 -> 512x7x7
        # trainable params: 0
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))

        # flatten : 512x7x7 -> 25088
        # 25088 -> 1000
        self.classifier = nn.Sequential(
            # 25088 -> 4096
            # trainable params: (25088 + 1) * 4096 = 102,764,544
            nn.Linear(25088, 4096),
            # 4096 -> 4096
            # trainable params: 0
            nn.ReLU(inplace=True),
            # 4096 -> 4096
            # trainable params: 0
            nn.Dropout(),
            # 4096 -> 4096
            # trainable params: (4096 + 1) * 4096 = 16,781,312
            nn.Linear(4096, 4096),
            # 4096 -> 4096
            # trainable params: 0
            nn.ReLU(inplace=True),
            # 4096 -> 4096
            # trainable params: 0
            nn.Dropout(),
            # 4096 -> 1000
            # trainable params: (4096 + 1) * 1000 = 4,097,000
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        # feature extractor
        x = self.features(x)

        # adaptive average pooling
        x = self.avgpool(x)

        # flatten : 512x7x7 -> 25088
        x = torch.flatten(x, start_dim=1)

        # classifier
        x = self.classifier(x)

        return x

    def _make_layers(self, cfg: list) -> list:
        layers = []
        in_channels = 3

        for x in cfg:
            if x == "M":
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1), nn.ReLU(inplace=True)]
                in_channels = x

        return layers

### <a id='toc2_1_1_'></a>[VGG11](#toc0_)


#### <a id='toc2_1_1_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg11_1 = CustomVGGNet(feature_layers=[64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"])

In [ ]:
vgg11_1

#### <a id='toc2_1_1_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg11_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_2_'></a>[VGG13](#toc0_)


#### <a id='toc2_1_2_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg13_1 = CustomVGGNet(feature_layers=[64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"])

In [ ]:
vgg13_1

#### <a id='toc2_1_2_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg13_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_3_'></a>[VGG16](#toc0_)


#### <a id='toc2_1_3_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg16_1 = CustomVGGNet(
    feature_layers=[64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
)

In [ ]:
vgg16_1

#### <a id='toc2_1_3_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg16_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_4_'></a>[VGG19](#toc0_)


#### <a id='toc2_1_4_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg19_1 = CustomVGGNet(
    feature_layers=[
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ]
)

In [ ]:
vgg19_1

#### <a id='toc2_1_4_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg19_1, (1, 3, 224, 224), device="cpu")

## <a id='toc2_2_'></a>[PyTorch VGGNet](#toc0_)

- All VGGNet variants available in PyTorch: [docs.pytorch.org/vision/stable/models/vgg.html](https://docs.pytorch.org/vision/stable/models/vgg.html)


### <a id='toc2_2_1_'></a>[VGG11](#toc0_)


#### <a id='toc2_2_1_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg11_2 = vgg11()

In [ ]:
vgg11_2

#### <a id='toc2_2_1_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg11_2, (1, 3, 227, 227), device="cpu")

### <a id='toc2_2_2_'></a>[VGG13](#toc0_)


#### <a id='toc2_2_2_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg13_2 = vgg13()

In [ ]:
vgg13_2

#### <a id='toc2_2_2_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg13_2, (1, 3, 227, 227), device="cpu")

### <a id='toc2_2_3_'></a>[VGG16](#toc0_)


#### <a id='toc2_2_3_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg16_2 = vgg16()

In [ ]:
vgg16_2

#### <a id='toc2_2_3_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg16_2, (1, 3, 227, 227), device="cpu")

### <a id='toc2_2_4_'></a>[VGG19](#toc0_)


#### <a id='toc2_2_4_1_'></a>[Initialize the Model](#toc0_)


In [ ]:
vgg19_2 = vgg19()

In [ ]:
vgg19_2

#### <a id='toc2_2_4_2_'></a>[Model Summary](#toc0_)


In [ ]:
summary(vgg19_2, (1, 3, 227, 227), device="cpu")